In [1]:
# 폴더 전체에서 PDF를 찾아 흑백(grayscale)으로 변환하여
# 같은 폴더에 '흑백-<원본이름>.pdf'로 저장합니다.

# 중복 방지 규칙:
# - 이미 '흑백-'으로 시작하는 파일은 건너뜀
# - 대상 폴더에 '흑백-<원본이름>.pdf'가 존재하면 건너뜀
# - 같은 경로를 한 번 이상 처리하지 않도록 실경로(realpath)로 중복 체크

# 권장: Ghostscript가 설치되어 있으면 벡터/텍스트가 보존된 흑백 PDF 생성.
# 미설치 시 PyMuPDF로 래스터 흑백 PDF 생성.


!apt -y install ghostscript        # (선택) 고품질 변환
!pip -q install pymupdf


import os
import sys
import subprocess
from typing import Optional

# -------- 설정 --------
ROOT_DIR = "."  # 시작 폴더 (예: "/content/drive/MyDrive/문서")
TARGET_PREFIX = "흑백-"  # 출력 파일 접두사
TARGET_GRAYSCALE_WIDTH_PX = 4096 # PyMuPDF 흑백 변환 시 목표 가로 픽셀 (래스터화)
# ----------------------

def which(cmd: str) -> Optional[str]:
    """PATH에서 실행파일 찾기 (Windows/Unix 호환)."""
    from shutil import which as _which
    return _which(cmd)

def get_gs_executable() -> Optional[str]:
    """
    Ghostscript 실행 파일 경로 탐지.
    - Linux/macOS: 'gs'
    - Windows: 'gswin64c' 또는 'gswin32c'
    """
    for cand in ("gs", "gswin64c", "gswin32c"):
        p = which(cand)
        if p:
            return p
    return None

def convert_with_ghostscript(gs: str, src: str, dst: str) -> None:
    """
    Ghostscript로 벡터 보존 흑백 변환.
    """
    cmd = [
        gs,
        "-sDEVICE=pdfwrite",
        "-dCompatibilityLevel=1.4",
        "-dProcessColorModel=/DeviceGray",
        "-sColorConversionStrategy=Gray",
        "-dColorConversionStrategy=/Gray",
        "-dDetectDuplicateImages",
        "-dNOPAUSE",
        "-dQUIET",
        "-dBATCH",
        f"-sOutputFile={dst}",
        src,
    ]
    subprocess.run(cmd, check=True)

def convert_with_pymupdf(src: str, dst: str) -> None:
    """
    PyMuPDF로 페이지를 회색조 이미지로 렌더링하여 새 PDF로 저장 (래스터화).
    텍스트/벡터는 이미지가 됨.
    """
    import fitz  # PyMuPDF
    doc = fitz.open(src)
    out = fitz.open()
    for i in range(len(doc)):
        p = doc.load_page(i)
        
        # 원본 페이지의 너비(포인트)를 기준으로 스케일 팩터 계산
        original_width_pts = p.rect.width
        # 원본 화질 유지를 위해, TARGET_GRAYSCALE_WIDTH_PX 보다 원본이 크면 원본 스케일 유지 (1.0), 작으면 확대
        scale_factor = max(1.0, TARGET_GRAYSCALE_WIDTH_PX / original_width_pts)
        matrix = fitz.Matrix(scale_factor, scale_factor) # 가로세로 비율 유지
        
        # 회색조 렌더링 (지정된 스케일 팩터 적용)
        pix = p.get_pixmap(matrix=matrix, colorspace=fitz.csGRAY)
        
        # 스케일된 이미지 크기에 맞춰 새 페이지 생성
        page = out.new_page(-1, width=pix.width, height=pix.height) # -1은 마지막에 추가
        page.insert_image(page.rect, stream=pix.tobytes("png"))
    out.save(dst)
    out.close()
    doc.close()

def convert_pdf_to_gray(src: str, dst: str, gs_path: Optional[str]) -> bool:
    """
    하나의 PDF를 흑백으로 변환. 성공 시 True.
    gs_path가 있으면 GS 사용, 없으면 PyMuPDF 사용.
    """
    try:
        if gs_path:
            convert_with_ghostscript(gs_path, src, dst)
        else:
            # PyMuPDF가 없을 수 있으므로 친절히 메시지
            try:
                convert_with_pymupdf(src, dst)
            except ModuleNotFoundError:
                raise RuntimeError(
                    "PyMuPDF가 설치되어 있지 않습니다. `pip install pymupdf` 후 다시 실행하세요."
                )
        return True
    except subprocess.CalledProcessError as e:
        print(f"[GS 실패] {src} -> {dst} : {e}", file=sys.stderr)
    except Exception as e:
        print(f"[변환 실패] {src} -> {dst} : {e}", file=sys.stderr)
    return False

def should_skip(src: str, dst: str) -> bool:
    """
    변환 필요 여부 판단:
    - 이미 흑백- 접두사로 시작하면 스킵
    - 출력 파일이 이미 존재하면 스킵
    """
    base = os.path.basename(src)
    if base.startswith(TARGET_PREFIX):
        return True
    if os.path.exists(dst):
        return True
    return False

def walk_and_convert(root: str) -> None:
    gs_path = get_gs_executable()
    if gs_path:
        print(f"[INFO] Ghostscript 발견: {gs_path} (벡터 보존 흑백 변환 사용)")
    else:
        print(f"[INFO] Ghostscript 미발견 \u2192 PyMuPDF 래스터 변환으로 진행 (목표 가로: {TARGET_GRAYSCALE_WIDTH_PX}px, 단 원본보다 작아지지 않음)")

    processed = set()  # realpath 기반 중복 방지

    total, converted, skipped = 0, 0, 0
    for dirpath, dirnames, filenames in os.walk(root):
        for name in filenames:
            if not name.lower().endswith(".pdf"):
                continue
            src = os.path.join(dirpath, name)
            rsrc = os.path.realpath(src)
            if rsrc in processed:
                continue
            processed.add(rsrc)

            dst = os.path.join(dirpath, f"{TARGET_PREFIX}{name}")
            total += 1

            if should_skip(src, dst):
                skipped += 1
                print(f"[SKIP] {src}")
                continue

            ok = convert_pdf_to_gray(src, dst, gs_path)
            if ok:
                converted += 1
                print(f"[OK]   {src} -> {dst}")
            else:
                print(f"[ERR]  {src}")

    print("\n--- 요약 ---")
    print(f"총 발견: {total}개  | 변환: {converted}개  | 스킵: {skipped}개")

if __name__ == "__main__":
    walk_and_convert(ROOT_DIR)

zsh:1: command not found: apt
[INFO] Ghostscript 미발견 → PyMuPDF 래스터 변환으로 진행
[OK]   ./흑백/유기화학 2/chapter 16_완.pdf -> ./흑백/유기화학 2/흑백-chapter 16_완.pdf
[OK]   ./흑백/유기화학 2/chapter 17_완.pdf -> ./흑백/유기화학 2/흑백-chapter 17_완.pdf
[OK]   ./흑백/유기화학 2/chapter 15_완.pdf -> ./흑백/유기화학 2/흑백-chapter 15_완.pdf
[OK]   ./흑백/유기화학 2/chapter 14_완.pdf -> ./흑백/유기화학 2/흑백-chapter 14_완.pdf
[OK]   ./흑백/유기화학 2/chapter 18_완.pdf -> ./흑백/유기화학 2/흑백-chapter 18_완.pdf
[OK]   ./흑백/유기화학 2/chapter 19_완.pdf -> ./흑백/유기화학 2/흑백-chapter 19_완.pdf
[OK]   ./흑백/재료열역학/(2025-2학기_열역학)Lecture_3.pdf -> ./흑백/재료열역학/흑백-(2025-2학기_열역학)Lecture_3.pdf
[OK]   ./흑백/재료열역학/(2025-2학기_열역학)Lecture_2.pdf -> ./흑백/재료열역학/흑백-(2025-2학기_열역학)Lecture_2.pdf
[OK]   ./흑백/재료열역학/(2025-2학기_열역학)Lecture_1.pdf -> ./흑백/재료열역학/흑백-(2025